# Profiling and Optimization

Profiling is essential for understanding application performance and identifying optimization opportunities.

Nvidia provides robust profiling tools suitable for all GPU programming approaches.

A common and effective strategy is the *top-down approach* to performance analysis: start with a *whole application* overview, then focus on specific *hot spots*.

Begin with Nsight Systems to get a broad performance overview, using either the command line or the GUI.
Then, use Nsight Compute to analyze the performance of individual kernels in detail.

## Nsight Systems Command Line Interface (CLI)

First, compile and run the benchmark application to ensure it produces correct results.

In [ ]:
!nvcc -O3 -std=c++17 -arch=sm_86 -o ../build/increase/increase-cuda-expl ../src/increase/increase-cuda-expl.cu

Next, profile the binary using `nsys profile`.

Key command line arguments:
* `--stats=true`: Prints a summary of performance statistics to the command line
* `-o ...`: Specifies the output profile file
* `--force-overwrite=true`: Overwrites the profile file if it already exists

In [ ]:
!nsys profile --stats=true -o ../profiles/increase-cuda-expl --force-overwrite=true ../build/increase/increase-cuda-expl

The most relevant sections in the profiling output are:

**CUDA API Statistics** (`cuda_api_sum`)

This section provides timing details for CUDA API calls such as `cudaMalloc`, `cudaMemcpy`, and `cudaDeviceSynchronize`.

For each API function, Nsight Systems reports:
* The relative and absolute total time spent
* The number of calls
* Statistics (min, max, average, median) for call durations

**CUDA Kernel Statistics** (`cuda_gpu_kern_sum`)

This section reports, for each kernel:
* The relative and absolute total execution time
* The number of launches
* Statistics (min, max, average, median) for kernel execution times

**Memory Transfers** (`cuda_gpu_mem_time_sum` and `cuda_gpu_mem_size_sum`)

These two sections focus on data transfers between host and device, reporting:
* The total time spent on transfers per direction, with statistics
* The total amount of data transferred, with statistics for individual transfers

## Nsight Systems Graphical User Interface (GUI)

To further investigate application behavior, open the generated `increase-cuda-expl.nsys-rep` report file in the `../profiles` folder **on your local PC**.

Download the file using the file browser on the left, or **shift** + **right-click** on this [link](../profiles/increase-cuda-expl.nsys-rep) and choose *Save Link As*.
Then, open the file with your local installation of Nsight Systems (version 2025.1.1 or newer required).

## Nsight Compute Command Line Interface (CLI)

After addressing system-level performance issues and identifying hot-spot kernels, the next step is to profile the application in detail using the Nsight Compute CLI (`ncu`).

Key command line arguments:
* `-o ...`: Specifies the output profile file (similar to `nsys`)
* `--force-overwrite`: Overwrites the profile file if it already exists (without `=true`, unlike `nsys`)

You can further limit which kernels are profiled with:
* `--launch-skip n` or `-s n`: Skips the first `n` kernels
* `--launch-count n` or `-c n`: Profiles only the first `n` applicable kernels
* `--kernel name` or `-k name`: Profiles only kernels with the specified name (supports regex and kernel renaming)

See the [documentation](https://docs.nvidia.com/nsight-compute/NsightComputeCli/index.html#profile) for a full list of arguments.

If no output file is specified, results are printed to the command line.

Note: If you do not restrict which kernels are profiled (e.g., by setting the launch count), *every kernel* will be profiled.

In [ ]:
!ncu -s 2 -c 1  ../build/increase/increase-cuda-expl

When profiling on a remote machine and analyzing results locally, it is often beneficial to collect more data than strictly necessary.
You can do this by adding the `--set=full` argument.

In [ ]:
!ncu -s 2 -c 1 --set=full -o ../profiles/increase-cuda-expl --force-overwrite ../build/increase/increase-cuda-expl

As before, download the resulting file (`increase-cuda-expl.ncu-rep`) using the file browser on the left or **shift** + **right-click** on this [link](../profiles/increase-cuda-expl.ncu-rep) and choose *Save Link As* to open it locally.

A comprehensive discussion of this topic is beyond this tutorial's scope.
For more details, see for example the NHR@FAU course *GPU Performance Engineering*.
The next workshop data will be announced at [https://hpc.fau.de/teaching/tutorials-and-courses](https://hpc.fau.de/teaching/tutorials-and-courses).

## Optimization Considerations

Using the performance evaluation techniques introduced above, you can identify different performance patterns.
These patterns not only guide optimization strategies, but also inform the choice of GPU programming approaches.

**Majority of Time Spent in Device Synchronization**

While this often means the GPU is doing useful work, it may also indicate that additional, independent CPU work could be overlapped.
Achieving this requires fully asynchronous kernel execution.

**Low Degree of Parallelism**

A single kernel may not fully utilize the GPU's capabilities.
This is often reflected in low *occupancy*, which can sometimes be estimated from hardware characteristics.
For example, if the number of CUDA blocks is less than the number of available SMs, or if block sizes are very small, parallelism is likely insufficient.

To address this, consider restructuring the computation to expose more parallelism, or run multiple kernels in parallel.
This may require modeling dependencies between kernels and using finer-grained synchronization.

**Data Transfers between Host and Device Dominate**

If your application uses managed memory, first consider applying prefetching.
If prefetching is not supported by your programming approach, switching to explicit memory management may help.

Another optimization is to overlap data transfers and kernel execution.
This requires fully asynchronous kernel execution, asynchronous memory transfers, and appropriate synchronization or dependency primitives.

**Data Sharing and Aggregation across Groups of Threads**

In many applications, threads within a group (block, team, gang, work-group, etc.) repeatedly access the same data elements.
While hardware caching often helps, manual buffering can sometimes be more efficient.

Using SM-local *shared memory* enables both efficient data sharing and synchronization among threads in a group.